In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5520/4151465537.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


In [3]:
df_bq=df_bq2[(df_bq2["GP"]=='Belgian Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4767/1686006020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
2166,0 days 00:57:12.422000,ALB,23,0 days 00:01:57.321000,1,1.0,NaT,NaT,NaT,0 days 00:00:52.352000,0 days 00:00:29.753000,NaT,0 days 00:56:42.783000,0 days 00:57:12.513000,333.0,198.0,221.0,297.0,False,MEDIUM,1.0,True,Williams,0 days 00:55:14.764000,NaT,1,10.0,False,,False,False,2024,Belgian Grand Prix
1808,0 days 00:59:03.705000,ALB,23,0 days 00:01:51.283000,2,1.0,NaT,NaT,0 days 00:00:31.152000,0 days 00:00:50.603000,0 days 00:00:29.528000,0 days 00:57:43.532000,0 days 00:58:34.135000,0 days 00:59:03.663000,NaN,196.0,219.0,314.0,True,MEDIUM,2.0,True,Williams,0 days 00:57:12.422000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1809,0 days 01:00:53.905000,ALB,23,0 days 00:01:50.200000,3,1.0,NaT,NaT,0 days 00:00:30.836000,0 days 00:00:49.980000,0 days 00:00:29.384000,0 days 00:59:34.499000,0 days 01:00:24.479000,0 days 01:00:53.863000,332.0,196.0,218.0,313.0,True,MEDIUM,3.0,True,Williams,0 days 00:59:03.705000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1810,0 days 01:02:44.215000,ALB,23,0 days 00:01:50.310000,4,1.0,NaT,NaT,0 days 00:00:31.057000,0 days 00:00:49.970000,0 days 00:00:29.283000,0 days 01:01:24.920000,0 days 01:02:14.890000,0 days 01:02:44.173000,325.0,196.0,218.0,NaN,False,MEDIUM,4.0,True,Williams,0 days 01:00:53.905000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix
1811,0 days 01:04:34.938000,ALB,23,0 days 00:01:50.723000,5,1.0,NaT,NaT,0 days 00:00:30.919000,0 days 00:00:50.405000,0 days 00:00:29.399000,0 days 01:03:15.092000,0 days 01:04:05.497000,0 days 01:04:34.896000,349.0,192.0,218.0,315.0,False,MEDIUM,5.0,True,Williams,0 days 01:02:44.215000,NaT,1,10.0,False,,False,True,2024,Belgian Grand Prix


In [4]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [5]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [6]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [7]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
2166,ALB,1,0.0,52.352,29.753,00:01:57.321000,117.321,MEDIUM,2024,Belgian Grand Prix,10.0,0.0,10.0,13.0
2162,ALO,1,0.0,51.662,29.580,00:01:56.577000,116.577,MEDIUM,2024,Belgian Grand Prix,8.0,0.0,8.0,9.0
2169,BOT,1,0.0,53.131,29.699,00:01:58.761000,118.761,MEDIUM,2024,Belgian Grand Prix,13.0,0.0,15.0,11.0
2167,GAS,1,0.0,52.344,29.936,00:01:58.554000,118.554,MEDIUM,2024,Belgian Grand Prix,12.0,0.0,9.0,18.0
2155,HAM,1,0.0,49.406,29.478,00:01:52.700000,112.700,MEDIUM,2024,Belgian Grand Prix,2.0,0.0,2.0,4.0


In [8]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [9]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,Belgian Grand Prix,2024,2,00:01:49.828000,109.828,VER,MEDIUM,30.617,NOR,MEDIUM,48.675,LEC,MEDIUM,29.350,NOR,MEDIUM
1,Belgian Grand Prix,2024,3,00:01:49.109000,109.109,HAM,MEDIUM,30.836,ALB,MEDIUM,48.486,HAM,MEDIUM,29.265,RUS,MEDIUM
2,Belgian Grand Prix,2024,4,00:01:49.790000,109.790,PER,MEDIUM,30.942,HUL,MEDIUM,48.605,HAM,MEDIUM,29.239,OCO,MEDIUM
3,Belgian Grand Prix,2024,5,00:01:49.376000,109.376,HAM,MEDIUM,30.860,PIA,MEDIUM,48.320,HAM,MEDIUM,29.294,RUS,MEDIUM
4,Belgian Grand Prix,2024,6,00:01:49.400000,109.400,HAM,MEDIUM,30.907,MAG,MEDIUM,48.355,HAM,MEDIUM,29.297,PIA,MEDIUM


In [10]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Belgian Grand Prix,2024,2,00:01:49.828000,109.828,VER,MEDIUM,30.617,NOR,MEDIUM,48.675,LEC,MEDIUM,29.350,NOR,MEDIUM,#3860C6
1,Belgian Grand Prix,2024,3,00:01:49.109000,109.109,HAM,MEDIUM,30.836,ALB,MEDIUM,48.486,HAM,MEDIUM,29.265,RUS,MEDIUM,#27F4D2
2,Belgian Grand Prix,2024,4,00:01:49.790000,109.790,PER,MEDIUM,30.942,HUL,MEDIUM,48.605,HAM,MEDIUM,29.239,OCO,MEDIUM,#3860C6
3,Belgian Grand Prix,2024,5,00:01:49.376000,109.376,HAM,MEDIUM,30.860,PIA,MEDIUM,48.320,HAM,MEDIUM,29.294,RUS,MEDIUM,#27F4D2
4,Belgian Grand Prix,2024,6,00:01:49.400000,109.400,HAM,MEDIUM,30.907,MAG,MEDIUM,48.355,HAM,MEDIUM,29.297,PIA,MEDIUM,#27F4D2


In [11]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
42,Belgian Grand Prix,2024,44,00:01:44.701000,104.701,PER,SOFT,30.498,PIA,HARD,45.309,PER,SOFT,28.584,PER,SOFT,#3860C6
29,Belgian Grand Prix,2024,31,00:01:45.563000,105.563,NOR,HARD,30.359,NOR,HARD,46.509,PIA,HARD,28.419,PIA,HARD,#FF8000
30,Belgian Grand Prix,2024,32,00:01:45.840000,105.840,PIA,HARD,30.450,OCO,HARD,46.432,PIA,HARD,28.577,OCO,HARD,#FF8000
40,Belgian Grand Prix,2024,42,00:01:46.262000,106.262,PIA,HARD,30.522,NOR,HARD,46.839,PIA,HARD,28.662,PIA,HARD,#FF8000
31,Belgian Grand Prix,2024,33,00:01:46.265000,106.265,PIA,HARD,30.640,PIA,HARD,46.607,HAM,HARD,28.740,OCO,HARD,#FF8000


In [12]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=11)
    ))
 )

# fig2.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": LapTime_ordered.T.values
#                                 if c == "All"
#                                 else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )

fig2.update_layout(
    title="Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="Black"
    ),
    height=780,
    width=1200,
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)


fig2.show()

In [13]:
fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))

In [ ]:
#Multi GP Plot

In [9]:
#2020
def driver_color(driver):
    if driver =="AIT":
        return "#0082fa"
    elif driver =="ALB":
        return '#1E41FF'
    elif driver =="BOT":
        return '#00D2BE'
    elif driver =="FIT":
        return '#787878'
    elif driver =="GAS":
        return '#2B4562'
    elif driver =="GIO":
        return '#9B0000'
    elif driver =="GRO":
        return '#787878'
    elif driver =="HAM":
        return '#00D2BE'
    elif driver =="HUL":
        return '#F596C8'
    elif driver =="KVY":
        return '#2B4562'
    elif driver =="LAT":
        return '#0082fa'
    elif driver =="LEC":
        return '#DC0000'
    elif driver =="MAG":
        return '#787878'
    elif driver =="NOR":
        return '#FF8700'
    elif driver =="OCO":
        return '#ffc600'
    elif driver =="PER":
        return '#F596C8'
    elif driver =="RAI":
        return '#F596C8'
    elif driver =="RIC":
        return '#ffc600'
    elif driver =="RUS":
        return '#0082fa'
    elif driver =="SAI":
        return '#FF8700'
    elif driver =="STR":
        return '#F596C8'
    elif driver =="VER":
        return '#1E41FF'
    elif driver =="VET":
        return '#DC0000'
    else:
        return '#3860C6'

In [10]:
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:

    df_bq=df_bq2[(df_bq2["GP"]==i)&(df_bq2["Year"]==year_multi)]
    GP=df_bq
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
    GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
    GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

    GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
    GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
    GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()
    GP_Rk=GP[GP["LapTime"]!="NaT"]
    GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
    GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
    GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)

    GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
    GP_Sectors=GP_Sectors.fillna(0)
    GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()
    Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
    Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
    Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
    Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
    Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
    Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
    Fast_Laps=Fast_Laps.reset_index(drop=True)
    Fast_Laps.head()
    Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
    Fast_Laps.head()
    LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
    LapTime_ordered.head()
    import plotly.graph_objects as go

    fig2 = go.Figure(go.Table
    (header={"values": LapTime_ordered.columns},
    cells=dict(
        values=LapTime_ordered.T.values,
        fill_color=[LapTime_ordered.color],
        align='center', font=dict(color='white', size=11)
        ))
    )
    fig2.update_layout(
        title="Fastest Lap and Sector Times Ordered By Lap Times For The {} {}".format(year_multi, i),
            title_x=0.5,
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="Black"
        ),
        height=780,
        width=1200,
        title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5),
    )
    fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5520/2004365406.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5520/2004365406.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5520/2004365406.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [14]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps.columns},
#   cells=dict(
#     values=Fast_Laps.T.values,
#     fill_color=[Fast_Laps.color],
#     align='center', font=dict(color='white', size=11)
#     ))
#  )

# # fig.update_layout(
# #     updatemenus=[
# #         {
# #             "buttons": [
# #                 {
# #                     "label": c,
# #                     "method": "update",
# #                     "args": [
# #                         {
# #                             "cells": {
# #                                 "values": Fast_Laps.T.values
# #                                 if c == "All"
# #                                 else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
# #                             }
# #                         }
# #                     ],
# #                 }
# #                 for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
# #             ]
# #         }
# #     ]
# # )
# fig.update_layout(
#     title="Fastest Lap and Sector Times For Ordered By Lap Number",
#         title_x=0.5,
#         # template="plotly_dark",

#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=13,
#         color="Black"
#     ),
#     height=780,
#     width=1200,
#     title_font_family="PT Sans Narrow",
#     margin=dict(l=5, r=5, t=35, b=5),
# )

# fig.show()

In [162]:
# fig.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [163]:
# #get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

# Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
# Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
# Fast_Laps_short.head()

In [164]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps_short.columns},
#   cells=dict(
#     values=Fast_Laps_short.T.values,
#     height=50,
#     fill_color=[Fast_Laps_short.color],
#     align='center', font=dict(color='white', size=20)
#     ))
#  )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": Fast_Laps_short.T.values
#                                 if c == "All"
#                                 else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )
# fig.update_layout(
#     title="Fastest Lap and Sector Times",
#         title_x=0.5,
#     yaxis = dict(tickfont = dict(size=20)),
#     xaxis = dict(tickfont = dict(size=20)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=20,
#         color="Black"
#     ),
#     height=2500,
#     title_font_family="PT Sans Narrow"
# )

# fig.show()

In [107]:
# #get a view for fastest lap in the entire race
# Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
# Fastest_Lap_Race

In [108]:
# #Multiple plotly filters code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "y": 1 - (i / 5),
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "restyle",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df[menu].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df[menu].unique().tolist()
#             ],
#         }
#         for i, menu in enumerate(["Channel_type", "Date"])
#     ]
# )

In [109]:
# #Single Plotly filter code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df["Channel_type"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df["Channel_type"].unique().tolist()
#             ]
#         }
#     ]
# )